In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import pymongo
import itertools
from concurrent.futures import ProcessPoolExecutor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
from sklearn.externals import joblib
from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
NASDAQ100 = CONF['NASDAQ100']

In [2]:
%%time
spider = Spider_nasdaq()
timeframe = '2y'
# for symbol in NASDAQ100:
for symbol in ['NDX']:
    datafile = spider.get_stock_data(symbol, timeframe)
    print(datafile)

../database/market/NDX.csv
Wall time: 2.71 s


In [3]:
%%time

# symbol = 'TSLA'

stock_df_dict = {}

for symbol in NASDAQ100:
    stock_data_file = '../database/market/%s.csv' % symbol
    stock_df = pd.read_csv(stock_data_file)

    # 筛选字段
#     stock_df = stock_df.loc[:, ['date', 'open', 'close', 'volume']]
    stock_df = stock_df.loc[:, ['date', 'close']]

    # 去掉Nasdaq行情首行的当天行情
    if symbol in NASDAQ:
        stock_df = stock_df.drop([0])

    # 处理Coinmarketcap的早期缺失数据
    if symbol in CRYPTOCURRENCY:
        stock_df = stock_df.replace(to_replace='-', value=pd.np.nan)

    # 抛弃空值异常值
    stock_df.dropna(axis=0, how='any', inplace=True)

    # 格式化日期，和news_df['publish_time']一致
    stock_df['date'] = stock_df['date'].apply(lambda x: pd.Period(x, freq='D'))

    # 转换字段格式
#     stock_df = stock_df.astype(dtype={'volume': 'float64'})

    # 用日期作索引，日期升序排序
    stock_df.set_index(['date'], inplace=True)
    stock_df = stock_df.sort_index(axis=0, ascending=True)

    stock_df['pct_change'] = stock_df.close.pct_change(1)
    
    stock_df = stock_df['2016-01-01':]
    
    stock_df_dict[symbol] = stock_df

# stock_df
# df = stock_df.copy()

Wall time: 46 s


In [4]:
%%time

PROPERTY = 100000
hold = []

# print(stock_df_dict['TSLA'].index)
# print(pd.period_range(start='2017-01-01', end='2018-01-01', freq='D'))

yesterday = None

for today in pd.period_range(start='2016-01-01', end='2018-05-01', freq='D'):
#     print(date)
    if today not in stock_df_dict['NDX'].index:
        continue
#     print(stock_df_dict['TSLA'].loc[date, :])

    for order in hold:
        if stock_df_dict[order[1]].loc[today, 'close'] > order[4] or stock_df_dict[order[1]].loc[today, 'close'] / order[4] < 0.75 or today - order[0] > 30:
            PROPERTY += stock_df_dict[order[1]].loc[today, 'close'] * order[2]
            hold.remove(order)
            print('%s SELL %s * %d @ %f' % (today, order[1], order[2], stock_df_dict[order[1]].loc[today, 'close']))
    
    for symbol in NASDAQ100:
#     for symbol in ['TSLA', 'NVDA', 'BIDU', 'FB']:
        if today not in stock_df_dict[symbol].index:
            continue
        if stock_df_dict[symbol].loc[today, 'pct_change'] < -0.05:
            if PROPERTY < 10000:
                continue
            buy = int(10000 / stock_df_dict[symbol].loc[today, 'close'])
#             buy = int(PROPERTY / 2 / stock_df_dict[symbol].loc[today, 'close'])
            PROPERTY -= stock_df_dict[symbol].loc[today, 'close'] * buy
            hold.append([today, symbol, buy, stock_df_dict[symbol].loc[today, 'close'], stock_df_dict[symbol].loc[yesterday, 'close']])
            print('%s BUY  %s * %d @ %f EXPECT %f' % (today, symbol, buy, stock_df_dict[symbol].loc[today, 'close'], stock_df_dict[symbol].loc[yesterday, 'close']))
    
    yesterday = today
            
print(PROPERTY)
print(hold)

PROPERTY = PROPERTY + sum([stock_df_dict[order[1]].loc[yesterday, 'close'] * order[2] for order in hold])

print(PROPERTY)

# s = 0
# c = 0
# for symbol in NASDAQ100:
# # for symbol in ['TSLA', 'NVDA', 'BIDU', 'FB']:
#     s_p = stock_df_dict[symbol].loc['2017-01-03', 'close']
#     e_p = stock_df_dict[symbol].loc['2018-01-03', 'close']
#     print(symbol, s_p, e_p, e_p / s_p)
#     s += e_p / s_p
#     c += 1

# print(s / c)
    
s_p = stock_df_dict['NDX'].loc['2016-06-01', 'close']
e_p = stock_df_dict['NDX'].loc['2018-05-01', 'close']
print('NDX', s_p, e_p, e_p / s_p)

2016-05-20 BUY  ROST * 190 @ 52.490000 EXPECT 55.520000
2016-06-07 BUY  ALXN * 72 @ 138.130000 EXPECT 154.990000
2016-06-07 BUY  BIIB * 42 @ 236.765100 EXPECT 271.391200
2016-06-07 BUY  JD * 454 @ 22.010000 EXPECT 23.320000
2016-06-08 BUY  CTRP * 238 @ 42.000000 EXPECT 44.300000
2016-06-10 BUY  NTES * 61 @ 161.980000 EXPECT 171.480000
2016-06-20 SELL ROST * 190 @ 54.200000
2016-06-22 BUY  ADBE * 106 @ 94.010000 EXPECT 99.720000
2016-06-22 BUY  TSLA * 50 @ 196.660000 EXPECT 219.610000
2016-06-23 SELL NTES * 61 @ 172.910000
2016-06-24 SELL ALXN * 72 @ 112.990000
2016-06-24 BUY  ALXN * 88 @ 112.990000 EXPECT 123.570000
2016-06-24 BUY  AAL * 369 @ 27.050000 EXPECT 30.330000
2016-06-24 BUY  ADI * 181 @ 55.040000 EXPECT 58.110000
2016-06-24 BUY  AMAT * 430 @ 23.230000 EXPECT 24.520000
2016-07-08 SELL BIIB * 42 @ 235.304400
2016-07-08 SELL ALXN * 88 @ 124.420000
2016-07-08 SELL AMAT * 430 @ 24.930000
2016-07-11 SELL JD * 454 @ 21.760000
2016-07-11 SELL TSLA * 50 @ 224.780000
2016-07-11 SELL A

2017-05-11 BUY  HOLX * 229 @ 43.620000 EXPECT 45.960000
2017-05-11 BUY  SYMC * 317 @ 31.460000 EXPECT 33.140000
2017-05-15 SELL FAST * 216 @ 44.200000
2017-05-17 BUY  AMAT * 233 @ 42.760000 EXPECT 45.070000
2017-05-18 SELL INCY * 79 @ 128.800000
2017-05-18 BUY  CSCO * 318 @ 31.380000 EXPECT 33.820000
2017-05-25 SELL KLAC * 101 @ 104.850000
2017-05-26 SELL ESRX * 166 @ 59.760000
2017-05-26 SELL AMAT * 233 @ 45.500000
2017-05-30 SELL STX * 238 @ 43.240000
2017-06-05 SELL ADP * 102 @ 101.100000
2017-06-05 SELL FOX * 352 @ 27.360000
2017-06-05 BUY  INCY * 80 @ 124.030000 EXPECT 131.500000
2017-06-06 SELL FOXA * 346 @ 27.530000
2017-06-06 BUY  FAST * 237 @ 42.100000 EXPECT 44.650000
2017-06-07 SELL VRSK * 130 @ 80.740000
2017-06-08 SELL LBTYA * 316 @ 29.860000
2017-06-09 BUY  AMAT * 223 @ 44.740000 EXPECT 47.430000
2017-06-09 BUY  CTXS * 127 @ 78.320000 EXPECT 83.210000
2017-06-09 BUY  JD * 256 @ 39.000000 EXPECT 41.920000
2017-06-09 BUY  KLAC * 98 @ 101.980000 EXPECT 108.930000
2017-06-09 

2018-02-27 SELL TXN * 91 @ 108.340000
2018-02-27 BUY  CMCSA * 272 @ 36.660000 EXPECT 39.580000
2018-02-27 BUY  ORLY * 41 @ 238.580000 EXPECT 254.970000
2018-02-28 SELL WYNN * 55 @ 167.500000
2018-02-28 BUY  CELG * 114 @ 87.120000 EXPECT 95.780000
2018-03-01 SELL WYNN * 61 @ 163.420000
2018-03-01 BUY  ALGN * 40 @ 245.550000 EXPECT 262.520000
2018-03-01 BUY  MNST * 184 @ 54.220000 EXPECT 63.370000
2018-03-02 SELL MXIM * 162 @ 61.230000
2018-03-02 BUY  JD * 228 @ 43.800000 EXPECT 46.210000
2018-03-05 SELL WBA * 131 @ 70.550000
2018-03-05 SELL PYPL * 127 @ 79.070000
2018-03-05 SELL GOOGL * 8 @ 1094.760000
2018-03-05 SELL SHPG * 78 @ 134.670000
2018-03-06 SELL CDNS * 236 @ 38.980000
2018-03-07 SELL REGN * 28 @ 334.410000
2018-03-07 SELL ALGN * 40 @ 262.900000
2018-03-07 BUY  DLTR * 112 @ 89.250000 EXPECT 104.360000
2018-03-07 BUY  ROST * 132 @ 75.400000 EXPECT 80.510000
2018-03-12 SELL EXPE * 96 @ 114.440000
2018-03-19 BUY  FB * 57 @ 172.560000 EXPECT 185.090000
2018-03-22 BUY  BIDU * 42 @ 